<div style="background-color:#7a1400;height:60px;padding-top:7px;">
<h3 align=center style="color:white">
Multivariate Sales Forecasting - German Retail Chain - Multivariate Forecasting
</h3> 
</div>

<h4>Avinash Pasupulate</h4><br>
avinash.pasupulate@gmail.com<br>
<a href="www.avinash.pasupulate.com">www.avinash.pasupulate.com</a>
<br><br>
<i>7th January 2019</i>

<br>
<hr>
<b>Open the html file in the repository or click 
<a href="http://nbviewer.jupyter.org/github/avinashpasupulate/multivariate_sales_forecasting_german_retail_chain/blob/master/sales_forecasting.ipynb">Nbconvert Notebook</a> to view plots that are properly rendered.
<br><br>
Please wait for the html page to load completely.</b>
<br>
<hr>

<h3>Methods for Multivariate Forecasting: </h3>
<hr>
<ul>
    <li> Linear Regression
    <li> ARIMAX - Auto-Regressive Integrated Moving Average with Exogenous variables
    <li> SARIMAX - Seasonal Auto-Regressive Integrated Moving Average with Exogenous variables
    <li> VAR - Vector AutoRegression
    <li> Prophet by Facebook
     </ul>
<br><br>       
 
 
<h3>Steps to Complete: </h3>
<hr>
<ul>
    <li> Clean-up & remove/replace null values

In [391]:
import os
import copy
import warnings
import random #setting seed

#data 
import datetime as dt
import pandas as pd

#visualization
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm._tqdm_notebook import tqdm_notebook

#data pre-processing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


% matplotlib inline
tqdm_notebook.pandas()
warnings.filterwarnings('ignore')
pd.set_option('float_format','{:f}'.format)

In [365]:
cwd = os.getcwd()
random.seed(9891482)

In [366]:
forecast_ads = pd.read_csv(cwd+r'/data/processed_data/sales_ads.csv')

<h3>Data Cleanup</h3>
<hr>
<ul>
    <li> Check for missing and partial values
    <li> Replace / correct missing or partial values
    <li> Remove rows containing NA values
    <li> Final Checks
</ul>

In [367]:
#making copy of the imported data
forecast_clean = copy.deepcopy(forecast_ads)

In [368]:
#checking for missing and partial information
num = forecast_clean.describe(include='all').T
num['count']=num['count'].astype('int64')
num[['count', 'unique']].sort_values('count', ascending=True)

count unique
CompetitionDistance     2611969    NaN
Unnamed: 0              2617643    NaN
days_since_competition  2617643   9461
Assortment              2617643      3
StoreType               2617643      4
relh                    2617643    NaN
tmpc                    2617643    NaN
cpi                     2617643    NaN
num_asy_app             2617643     31
wc_days                 2617643    NaN
google_svi              2617643    NaN
mapped_region           2617643     12
quarter                 2617643    NaN
days_since_promo2       2617643   2183
seasons                 2617643      4
weekday                 2617643      7
iso_weeknum             2617643    NaN
day                     2617643    NaN
year                    2617643    NaN
SchoolHoliday           2617643    NaN
StateHoliday            2617643      4
Promo                   2617643    NaN
Open                    2617643    NaN
Customers               2617643    NaN
Sales                   2617643    NaN
date_s                  2617643    942
Store                   2617643    NaN
month                   2617643     12
promo2_int_mod          2617643    NaN

In [369]:
num[['count', 'unique']][num['unique']<10]

count unique
StateHoliday  2617643      4
weekday       2617643      7
seasons       2617643      4
StoreType     2617643      4
Assortment    2617643      3

In [370]:
#replacing na values in the CompetitionDistance column with 0 
forecast_clean.loc[forecast_clean['CompetitionDistance'].isna(),
                   'CompetitionDistance']=0

In [371]:
#replace timedelta values with numbers in days_since_competition and days_since_promo2 columns
forecast_clean['days_since_competition'] = pd.to_timedelta(forecast_clean['days_since_competition']).dt.days
forecast_clean['days_since_promo2'] = pd.to_timedelta(forecast_clean['days_since_promo2']).dt.days


In [372]:
#list all datatypes in forecast_clean dataframe
forecast_clean.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2617643 entries, 0 to 2617642
Data columns (total 29 columns):
Unnamed: 0                int64
Store                     int64
date_s                    object
Sales                     int64
Customers                 int64
Open                      int64
Promo                     int64
StateHoliday              object
SchoolHoliday             int64
year                      int64
day                       int64
iso_weeknum               int64
weekday                   object
month                     object
seasons                   object
quarter                   int64
mapped_region             object
google_svi                int64
wc_days                   int64
num_asy_app               object
cpi                       float64
tmpc                      float64
relh                      float64
StoreType                 object
Assortment                object
CompetitionDistance       float64
days_since_competition    int64
days_s

In [373]:
#convert datatypes
forecast_clean['Store'] = forecast_clean['Store'].astype(str)
forecast_clean.loc[forecast_clean['StateHoliday']!='0', 'StateHoliday'] = 1
forecast_clean['SchoolHoliday'] = forecast_clean['SchoolHoliday'].astype(int)
forecast_clean['StateHoliday'] = forecast_clean['StateHoliday'].astype(int)
forecast_clean['num_asy_app'] = (forecast_clean['num_asy_app'].apply(lambda x: x.replace(',','').replace('.',''))).astype(float)
#remove date columns since they are captured separately
forecast_clean.drop('date_s', inplace=True, axis = 1)

In [384]:
#datetime object can also be converted to ordinal values line "701234" using the below code
#pd.to_datetime(forecast_ads['date_s']).map(dt.datetime.toordinal)

In [374]:
#converting date to ordinal
#removing store type and assortment since these variables are at the store level
#also excluding season because temperature and humidity can be used to simulate the same
weekdaymap = {'Friday':6, 'Thursday':5, 'Wednesday':4, 'Tuesday':3, 'Monday':2, 'Sunday':1,
       'Saturday':7}
forecast_clean['month'] = forecast_clean['month'].apply(lambda x: int(dt.datetime.strptime(x, '%B').month))
forecast_clean['weekday'] = forecast_clean['weekday'].map(weekdaymap).astype(int)

forecast_clean.drop(['StoreType', 'Assortment','seasons'], inplace=True, axis=1)


<h3>Linear Regression</h3>
<hr>
<ul>
    <li> One Hot Encoding
    <li> Train, Validation & Test Split
    <li> Feature Selection & run model
        <ul>
            <li> Backward Feature Selection
            <li> Forward Feature Selection
            <li> Stepwise Feature Selection
        </ul>
    <li> Validate model using validation dataset
    <li> Optimize Parameters to get an ideal fit for validation dataset
    <li> Test with test dataset
    <li> Intepret results
</ul>
<br>
<h4>Notes:</h4>
<ul>
    <li> Maynot be an ideal method to forecast a timeseries
    <li> Check from multi-collinearity
    <li> Check for missing or incomplete data
    <li> Check for outliers
    <li> Ensure data splits are made in sequence since this for a time series
</ul>
<br>
<h4>Reference:</h4> 
<ul>
    <li><a href="https://www.theanalysisfactor.com/13-steps-regression-anova/">13 Steps for Regression ANOVA</a>

In [375]:
#creating copy for linear regression
linear_ds = copy.deepcopy(forecast_clean)
linear_ds.head()

Unnamed: 0 Store  Sales  Customers  Open  Promo  StateHoliday  \
0           0     1   5263        555     1      1             0   
1           1     1   5263        555     1      1             0   
2           2     1   5020        546     1      1             0   
3           3     1   5020        546     1      1             0   
4           4     1   4782        523     1      1             0   

   SchoolHoliday  year  day       ...        google_svi  wc_days  num_asy_app  \
0              1  2015   31       ...                51        0 39825.000000   
1              1  2015   31       ...                51        0 39825.000000   
2              1  2015   30       ...                51        0 39825.000000   
3              1  2015   30       ...                51        0 39825.000000   
4              1  2015   29       ...                51        0 39825.000000   

         cpi      tmpc      relh  CompetitionDistance  days_since_competition  \
0 100.288454 16.296894 54.272174          1270.000000                    2524   
1 100.288454 16.296894 54.272174          1270.000000                    2524   
2 100.288454 15.944318 56.774830          1270.000000                    2523   
3 100.288454 15.944318 56.774830          1270.000000                    2523   
4 100.288454 16.233511 58.021436          1270.000000                    2522   

   days_since_promo2  promo2_int_mod  
0                  0               0  
1                  0               0  
2                  0               0  
3                  0               0  
4                  0               0  

[5 rows x 25 columns]

In [390]:
#testing with one store
def store_set(linear_ds, store):
    #subsetting data by store
    store1 = linear_ds.loc[linear_ds['Store']==store]
    store1.drop(['Store','Unnamed: 0','mapped_region'], inplace=True, axis=1)
    #One Hot Encoding
    store1 = pd.get_dummies(store1)
    return store1

In [386]:
store='1'
store1 = store_set(linear_ds, store)
len(store1)

1884

In [387]:
store1.columns

Index(['Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'year', 'day', 'iso_weeknum', 'weekday', 'month', 'quarter',
       'google_svi', 'wc_days', 'num_asy_app', 'cpi', 'tmpc', 'relh',
       'CompetitionDistance', 'days_since_competition', 'days_since_promo2',
       'promo2_int_mod'],
      dtype='object')

In [388]:
len(store1.columns)

22

<h3>Train, Validation & Test Split</h3><hr>

In [401]:
#splitting the data set into train and test dataset
train, test = train_test_split(store1, test_size=0.2)
train, valid = train_test_split(train, test_size=0.1)

In [414]:
print("Length of train set:\t {}\nLength of validation set: {}\nLength of test set:\t {}".format(len(train), len(valid), len(test)))

Length of train set:	 1356
Length of validation set: 151
Length of test set:	 377


In [442]:
x_train = train['Sales']
y_train = train[['Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'year', 'day', 'iso_weeknum', 'weekday', 'month', 'quarter',
       'google_svi', 'wc_days', 'num_asy_app', 'cpi', 'tmpc', 'relh',
       'CompetitionDistance', 'days_since_competition', 'days_since_promo2',
       'promo2_int_mod']]



There is not feature selection such as Forward, Backdard and stepwise, other options are to be explored

In [ ]:
#linear regression


<div style="background-color:#7a1400;height:40px;padding-top:0.1px;">
<h6 align=center style="color:white">
<a href="https://www.linkedin.com/in/avinashpasupulate/" style="color:white">Avinash Pasupulate - Linkedin Profile</a>
</h6> 
</div>